# Implementation of a Vanilla GAN

In [1]:
# Importing Libraries

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import *
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
batch_size = 128
X_dim = 784
Z_dim = 100

In [3]:
# Function to plot grids of images generated

def plot(samples):
    fig  = plt.figure(figsize = (4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace = 0.05,hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig
        

In [4]:
def xavier_init(dimension):
    in_dim = dimension[0]
    xavier_stddev = 1. / (tf.sqrt(in_dim / 2.))
    return tf.random_normal(shape = dimension, stddev = xavier_stddev)

def sample_z(m,n):
    return np.random.uniform(-1., 1., size =[m,n])

In [5]:
#Setting up weights and biases, as well as declaring inputs X and Z to discriminator and generato 


X = tf.placeholder(tf.float32,shape = [None, X_dim])

Z = tf.placeholder(tf.float32,shape = [None,Z_dim])

G_Weights1 = tf.Variable(xavier_init([Z_dim,128]))

G_Bias1 = tf.Variable(tf.zeros(shape = [128]))

G_Weights2 = tf.Variable(xavier_init([128,X_dim]))

G_Bias2 = tf.Variable(tf.zeros(shape=[X_dim]))

D_Weights1 = tf.Variable(xavier_init([X_dim,128]))

D_Bias1 = tf.Variable(tf.zeros(shape=[128]))

D_Weights2 = tf.Variable(xavier_init([128,1]))

D_Bias2 = tf.Variable(tf.zeros(shape = [1]))

theta_D = [D_Weights1,D_Weights2,D_Bias1,D_Bias2]
theta_G = [G_Weights1,G_Weights2,G_Bias1,G_Bias2]

In [6]:
def generator(z):
    G_Activation1 = tf.nn.relu(tf.matmul(z,G_Weights1)+G_Bias1)
    G_Activation2 = tf.matmul(G_Activation1,G_Weights2)+G_Bias2
    generated  = tf.nn.sigmoid(G_Activation2)
    return generated

In [7]:
def discriminator(X):
    D_Activation1 = tf.nn.relu(tf.matmul(X,D_Weights1)+D_Bias1)
    D_Logit = tf.matmul(D_Activation1,D_Weights2) + D_Bias2
    D_Prob = tf.nn.sigmoid(D_Logit)
    return D_Prob

In [8]:
generated_sample = generator(Z)

D_Real = discriminator(X)
D_Fake = discriminator(generated_sample)

D_loss = -tf.reduce_mean(tf.log(D_Real) + tf.log(1. - D_Fake))
G_loss = -tf.reduce_mean(tf.log(D_Fake))
D_optimiser = tf.train.AdamOptimizer().minimize(D_loss,var_list = theta_D) 
G_optimiser = tf.train.AdamOptimizer().minimize(G_loss,var_list = theta_G) 

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
i=0
for it in range(45000):
    X_mb, _ = mnist.train.next_batch(batch_size)
    Z_mb = sample_z(batch_size,Z_dim)
    _, D_loss_cur = sess.run([D_optimiser,D_loss], feed_dict ={X:X_mb, Z: Z_mb})
    _, G_loss_cur = sess.run([G_optimiser,G_loss],feed_dict = {Z:Z_mb})
    if it % 5000 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_cur, G_loss_cur))
    if it % 10000 == 0:
        samples = sess.run(generated_sample,feed_dict={Z: sample_z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('Vanilla_GAN_Images/{}.png'.format(str(i).zfill(3)),bbox_inches='tight')
        i+=1
        plt.close(fig)
    

Iter: 0; D_loss: 1.648; G_loss: 2.236
Iter: 5000; D_loss: 0.5516; G_loss: 3.937
Iter: 10000; D_loss: 0.5541; G_loss: 3.53
Iter: 15000; D_loss: 0.6824; G_loss: 2.294
Iter: 20000; D_loss: 0.9162; G_loss: 1.895
Iter: 25000; D_loss: 0.6352; G_loss: 2.23
Iter: 30000; D_loss: 0.7801; G_loss: 2.03


KeyboardInterrupt: 